## 1. This program will open https://www.redfin.com/ and search for the california counties
## 2. It will download the housing information of each county if the download link is available
## 3. It will finally merge all the housing information into one csv file
### Note: Install the below python libraries before running this notebook

#### pip install selenium
#### pip install requests
#### pip install bs4
#### pip install webdriver-manager
#### pip install pandas
#### pip install requests

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import requests
import io
import os
import glob

# Start the chrome browser
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
driver.maximize_window()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\avadavelli\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
C:\ProgramData\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


In [2]:
# Download the zip code database from https://www.unitedstateszipcodes.org/zip-code-database/
# Get all California county names
df = pd.read_csv('zip_code_database.csv')
df = df[df['state']=='CA']
df = df[df['county'].notna()]
all_ca_counties = df['county'].unique()
all_ca_counties

array(['Los Angeles County', 'Orange County', 'Ventura County',
       'San Bernardino County', 'Riverside County', 'San Diego County',
       'Imperial County', 'Inyo County', 'Santa Barbara County',
       'Tulare County', 'Kings County', 'Kern County', 'Fresno County',
       'San Luis Obispo County', 'Monterey County', 'Mono County',
       'Madera County', 'Merced County', 'Mariposa County',
       'San Mateo County', 'Santa Clara County', 'San Francisco County',
       'Sacramento County', 'Alameda County', 'Napa County',
       'Contra Costa County', 'Solano County', 'Marin County',
       'Sonoma County', 'Santa Cruz County', 'San Benito County',
       'San Joaquin County', 'Calaveras County', 'Tuolumne County',
       'Stanislaus County', 'Mendocino County', 'Lake County',
       'Humboldt County', 'Trinity County', 'Del Norte County',
       'Siskiyou County', 'Amador County', 'Placer County', 'Yolo County',
       'El Dorado County', 'Alpine County', 'Sutter County',
      

In [3]:
# This method will open https://www.redfin.com, search for the county name and return the download link
def searchByCounty(countyName):
    driver.get('https://www.redfin.com')
    time.sleep(15)
    try:
        driver.find_element(By.XPATH, "//input[contains(@id, 'search-box-input') and contains(@title, 'City, Address, School, Agent, ZIP')]").click()
        if driver.find_element(By.XPATH, "//input[contains(@title, 'Clear')]").is_displayed():
            driver.find_element(By.XPATH, "//input[contains(@title, 'Clear')]").click()
        driver.find_element(By.XPATH, "//input[contains(@id, 'search-box-input') and contains(@placeholder, 'City, Address, School, Agent, ZIP')]").send_keys(countyName)
        time.sleep(3)
        driver.find_element(By.XPATH, "//a[text()='" + countyName + "']").click()
        download_link = driver.find_element(By.XPATH, "//a[contains(@id, 'download-and-save')]").get_attribute("href")
        all_links = download_link.split('num_homes=350')
        print('Download link available for: '+countyName)
        return (all_links[0]+'num_homes=10000'+all_links[1])
    except Exception as e:
        print('Unable to get download link for: '+countyName)
        return      

In [4]:
# Remove None from the download_urls
download_urls = [searchByCounty(county) for county in all_ca_counties]
urls = list(filter(None, download_urls))

Download link available for: Los Angeles County
Download link available for: Orange County
Download link available for: Ventura County
Download link available for: San Bernardino County
Download link available for: Riverside County
Download link available for: San Diego County
Unable to get download link for: Imperial County
Unable to get download link for: Inyo County
Download link available for: Santa Barbara County
Unable to get download link for: Tulare County
Download link available for: Kings County
Download link available for: Kern County
Unable to get download link for: Fresno County
Download link available for: San Luis Obispo County
Download link available for: Monterey County
Unable to get download link for: Mono County
Unable to get download link for: Madera County
Unable to get download link for: Merced County
Unable to get download link for: Mariposa County
Download link available for: San Mateo County
Download link available for: Santa Clara County
Download link availabl

In [5]:
# Close the browser
driver.close()

In [6]:
# This method will download the refin data and save it to the csv file
def downloadCSV(url, index):
    headers = {"accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding" : "gzip, deflate, br", 
    "accept-language" : "en-US,en;q=0.9", 
    "cache-control" : "max-age=0",
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
    response = requests.get(url, headers=headers)
    if response.ok:
        data = response.content.decode('utf8')
        df = pd.read_csv(io.StringIO(data))
        if not os.path.exists('CountyData'):
            os.mkdir('CountyData')
        df.to_csv('CountyData/'+str(index)+'.csv')   

In [7]:
# Download the redfin data for each county
for idx, url in enumerate(urls):
    downloadCSV(url, idx)    

In [8]:
# Merge the redfin data of all counties and save it as .csv file 
path = r'CountyData'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[:-1]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('CountyData/AllCounties_Data.csv')